In [21]:
import os
import csv
import logging

from dotenv import load_dotenv
from llama_index.llms.openai import OpenAI
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.core.tools import FunctionTool
import nest_asyncio


def configure_logger():
    #logger, formatter = None, None
    logger = logging.getLogger("invoice_logger")
    logger.setLevel(logging.DEBUG)
    formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
    file_handler = logging.FileHandler("invoice_debug.log", mode="w")
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)
    console_handler = logging.StreamHandler()
    console_handler.setFormatter(formatter)
    logger.addHandler(console_handler)

    return logger


# === Configure Logging (file + console) ===
logger = configure_logger()

# === Set your OpenAI API Key ===
load_dotenv()

# === Tool: Read CSV Invoice ===
def read_invoice(file_path: str) -> list:
    """Reads a CSV invoice file. Returns list of 'Formatted Date : Quantity' strings."""
    invoice_data = []
    logger.info(f"Reading invoice file: {file_path}")

    try:
        with open(file_path, newline='') as csvfile:
            reader = csv.reader(csvfile)
            header = next(reader)
            logger.debug(f"CSV header: {header}")

            for row_num, row in enumerate(reader, start=1):
                if not row:
                    logger.warning(f"Skipping empty row {row_num}")
                    continue

                try:
                    raw_date = row[0].strip()
                    parsed_date = datetime.strptime(raw_date, "%m/%d/%Y")
                    formatted_date = parsed_date.strftime("%a, %b %-d")

                except Exception as e:
                    logger.error(f"Date parsing failed on row {row_num}: {row[0]} — {e}")
                    #formatted_date = f"Invalid date ({row[0]})"
                    formatted_date = raw_date

                time_spent = row[-2].strip()
                price_per_unit = row[-1].strip()
                logger.debug(f"Parsed row {row_num}: {formatted_date} : {time_spent} : {price_per_unit}")
                invoice_data.append(f"{formatted_date} : {time_spent} : {price_per_unit}")

    except FileNotFoundError:
        logger.error("File not found.")
        return ["ERROR: File not found."]
    except Exception as e:
        logger.exception(f"Unexpected error while reading file: {e}")
        return [f"ERROR: {str(e)}"]

    logger.info(f"Successfully read {len(invoice_data)} invoice rows.")
    return invoice_data

# === Tool: Generate PDF Invoice ===

from fpdf import FPDF
from datetime import datetime, timedelta

# === Supported Currency Symbols ===
CURRENCY_SYMBOLS = {
    "usd": "US$",
    "eur": "€",
    "gbp": "£",
    "ngn": "NGN",
    "jpy": "¥"
}
def get_currency_symbol(code: str) -> str:
    return CURRENCY_SYMBOLS.get(code.lower(), "US$")

def generate_invoice_pdf_with_reference(
    entries: list,
    output_path="styled_invoice.pdf",
    invoice_number: int = 13,
    logo_path: str = None,
    tax: float = 0,
    due_date: int = 7,
    sender_address: str = "No 14,\nIbadan",
    sender_name : str = "Olotin Emmanuel",
    recipient_name: str = "Recipient Naame",
    recipient_address: str = "Recipient Address\nState Country Postcode",
    currency: str = "usd"

) -> dict:
    """Generate a styled invoice PDF with optional company logo that spans full page width """
    logger.debug(f"Entries received: {entries}")

    if not entries or not isinstance(entries, list):
        logger.error("No valid entries provided for invoice.")
        return {
            "status": "Error",
            "message": "No valid invoice entries provided.",
            "pdf_path": None
        }

    today = datetime.today()
    due_date = today + timedelta(days=due_date)
    today_str = today.strftime("%b %d, %Y")
    due_date_str = due_date.strftime("%b %d, %Y")

    pdf = FPDF()
    pdf.add_page()

    page_width = pdf.w - 2 * pdf.l_margin

    col_item = page_width * 0.5
    col_qty = page_width * 0.15
    col_rate = page_width * 0.15
    col_amount = page_width * 0.15

    currency_symbol = get_currency_symbol(currency)

    # === Logo
    if logo_path and os.path.exists(logo_path):
        try:
            # Insert logo
            pdf.image(logo_path, x=10, y=10, w=25)
        except Exception as e:
            logger.warning(f"Failed to embed logo: {e}")
    # === Header
    pdf.set_font("Arial", "B", 20)
    pdf.cell(0, 5, "INVOICE", ln=True, align="R")
    pdf.set_font("Arial", "B", 12)
    pdf.cell(0, 10, f"# {invoice_number}", ln=True, align="R")
    pdf.set_font("Arial", "", size=10)
    pdf.cell(0, 10, f"Date: {today_str}", ln=True, align="R")
    pdf.cell(0, 10, f"Due Date: {due_date_str}", ln=True, align="R")

    right_x = pdf.l_margin + col_item + col_qty

    balance_box_x = None
    balance_box_y = None

    # Draw "Balance Due" box just under INVOICE title (Page 1 only)
    if pdf.page_no() == 1:
        box_x = pdf.w - pdf.r_margin - 60  # align to right margin
        box_y = pdf.get_y()  # just below "INVOICE"
        box_w = 60
        box_h = 10

        balance_box_x = box_x + box_w - 5
        balance_box_y = box_y + 5

        # Background box
        pdf.set_xy(box_x, box_y)
        pdf.set_fill_color(240, 240, 240)
        pdf.set_font("Arial", "B", 12)
        pdf.cell(box_w, box_h, "", 0, 1, "R", fill=True)

        # Left label
        pdf.set_xy(box_x + 2, box_y + 2)
        pdf.set_font("Arial", "B", 12)
        pdf.cell(0, 5, "Balance Due:", 0, 0, "L")


    # === Sender
    pdf.set_xy(10, 40)
    pdf.set_font("Arial", "B", 11)
    pdf.cell(0, 5, sender_name, ln=True)
    pdf.set_font("Arial", size=10)
    pdf.multi_cell(0, 5, sender_address)

    # === Recipient
    pdf.ln(5)
    pdf.set_font("Arial", "B", 10)
    pdf.cell(0, 6, "Bill To:", ln=True)
    pdf.set_font("Arial", "B", 11)
    pdf.cell(0, 5, recipient_name, ln=True)
    pdf.set_font("Arial", "", 10)
    pdf.multi_cell(0, 5, recipient_address)

    pdf.ln(10)

    # === Table Header
    pdf.set_font("Arial", "B", 12)
    pdf.set_fill_color(50, 50, 50)
    pdf.set_text_color(255, 255, 255)

    pdf.cell(col_item, 10, "Item", 1, 0, "L", True)
    pdf.cell(col_qty, 10, "Quantity", 1, 0, "C", True)
    pdf.cell(col_rate, 10, "Rate", 1, 0, "C", True)
    pdf.cell(col_amount, 10, "Amount", 1, 1, "R", True)

    # === Table Rows
    pdf.set_font("Arial", size=11)
    pdf.set_text_color(0, 0, 0)

    total_cost = 0
    valid_rows = 0

    for i, entry in enumerate(entries):
        # Handle string format: "Wed, Apr 2 : 4.4 : 50"
        if isinstance(entry, str) and ':' in entry:
            try:
                item, quantity_str, unit_price = map(str.strip, entry.split(':'))
                quantity = float(quantity_str)
                price_per_unit = float(unit_price)
                amount = quantity * price_per_unit
                total_cost += amount
                valid_rows += 1
            except Exception as e:
                logger.warning(f"Failed to parse entry at {i}: {entry} — {e}")
                continue
        elif isinstance(entry, dict) and 'date' in entry and 'quantity' in entry:
            item = entry['date']
            quantity = entry['quantity']
            price_per_unit = entry['price']

            amount = quantity * price_per_unit
            total_cost += amount
            valid_rows += 1

        elif isinstance(entry, dict) and 'item' in entry and 'quantity' in entry:
            item = entry['item']
            quantity = entry['quantity']
            price_per_unit = entry['price']

            amount = quantity * price_per_unit
            total_cost += amount
            valid_rows += 1
        else:
            logger.warning(f"Skipping malformed entry at {i}: {entry}")
            continue

        valid_rows += 1

        pdf.cell(col_item, 10, item, 1)
        pdf.cell(col_qty, 10, f"{quantity:.2f}", 1, 0, "C")
        pdf.cell(col_rate, 10, f"{currency_symbol}{price_per_unit:.2f}", 1, 0, "C")
        pdf.cell(col_amount, 10, f"{currency_symbol}{amount:.2f}", 1, 1, "R")

    if valid_rows == 0:
        return {
            "status": "Error",
            "message": "No valid rows to display.",
            "pdf_path": None
        }

    tax_amount = total_cost * (tax / 100)

    # === Totals (aligned right)
    pdf.ln(5)

    pdf.set_font("Arial", "", 11)
    pdf.set_xy(right_x, pdf.get_y())
    pdf.cell(col_rate, 8, "Subtotal:", 0, 0, "R")
    pdf.cell(col_amount, 8, f"{currency_symbol}{total_cost:.2f}", 0, 1, "R")

    pdf.set_x(right_x)
    pdf.cell(col_rate, 8, f"Tax ({tax}%):", 0, 0, "R")
    pdf.cell(col_amount, 8, f"{currency_symbol}{tax_amount}", 0, 1, "R")

    grand_total = total_cost - tax_amount

    pdf.set_font("Arial", "B", 12)
    pdf.set_x(right_x)
    pdf.cell(col_rate, 10, "Total:", 0, 0, "R")
    pdf.cell(col_amount, 10, f"{currency_symbol}{grand_total:.2f}", 0, 1, "R")

    current_page = pdf.page
    if balance_box_x and balance_box_y:
        pdf.page = 1
        pdf.set_xy(balance_box_x, balance_box_y)
        pdf.set_font("Arial", "B", 12)
        pdf.set_text_color(0, 0, 0)
        pdf.cell(0, 0, f"US${grand_total:.2f}", 0, 0, "R")
    else:
        logger.warning("Balance box coordinates not set, skipping amount overlay.")

    # Return to where we left off
    pdf.page = current_page
    # Save PDF
    pdf.output(output_path)
    logger.info(f"PDF saved to {output_path} — Total: ${total_cost - tax_amount:.2f}")

    return {
        "pdf_path": output_path,
        "total_cost": total_cost,
        "invoice_number": invoice_number,
        "status": "Invoice generated successfully."
    }


# === Wrap Functions as Tools ===
read_invoice_tool = FunctionTool.from_defaults(fn=read_invoice)
generate_pdf_tool = FunctionTool.from_defaults(fn=generate_invoice_pdf_with_reference)

# === Define the Agent ===
agent = FunctionAgent(
    tools=[read_invoice_tool, generate_pdf_tool],
    llm=OpenAI(model="gpt-4o-mini"),
    system_prompt = """
        You are an invoice assistant.

        You can use two tools:
        - `read_invoice`: to read invoice entries from a CSV file
        - `generate_invoice_pdf_with_reference`: to generate a styled PDF invoice

         Your job:
        - If entries are provided directly in the user's message, you may skip calling `read_invoice`.
        - Use `generate_invoice_pdf_with_reference` to create the PDF.
        - Pass all provided arguments (like logo_path, due_date, tax, entries, invoice number prefix, currency etc.) to the tool if mentioned by the user.

         STOP after:
        - Calling generate_invoice_pdf_with_reference once, no repeated function calls.
        - You get a successful response from `generate_invoice_pdf_with_reference` that includes a `status` field set to `Invoice generated successfully.`.

         Never re-call tools once the invoice is generated.
         End the conversation with the invoice summary and stop.
    """,
     max_iterations=1,  # prevent runaway loops
)

# === Run the Agent ===
nest_asyncio.apply()

async def main():
    try:
        response = await agent.run("""
        Use data from data/april_2025.csv.

        Also:
        - Use data/company_logo.png as the company logo.
        - Apply a tax of 0% to the total cost.
        - Due date is 7 days from today.
        - The invoice currency should be in Nigerian naira.
        - Generate the invoice number starting with INV_ followed by a random 6-digit number.
        - Use the following entries instead of the `read_invoice_tool`:
        [
          'Printer Ink Cartridge : 4.4 : 67',
          'Wireless Mouse : 5.0 : 42',
          'Desk Organizer : 5.3 : 88',
          'Ergonomic Chair : 1.5 : 115',
          'Whiteboard Markers : 1.1 : 34',
          'Sticky Notes : 2.6 : 22',
          'Laptop Stand : 2.7 : 95',
          'Filing Cabinet : 6.5 : 103',
          'Monitor Riser : 2.1 : 39',
          'Conference Speaker : 2.5 : 78',
          'Keyboard Tray : 2.7 : 65',
          'Surge Protector : 5.7 : 49',
          'USB-C Hub : 2.1 : 59',
          'Paper Shredder : 7.2 : 102',
          'Portable Scanner : 4.9 : 74',
          'Desk Lamp : 2.0 : 56',
          'Office Chair Mat : 7.4 : 81',
          'Document Tray : 3.6 : 33',
          'Wireless Keyboard : 4.8 : 60',
          'Notebook Pack : 4.9 : 19'
        ]
        """)

        print(response)
    except Exception as e:
        logger.exception("Error during agent execution")
        print(f"Error: {e}")

await main()


The invoice has been successfully generated. Here are the details:

- **Invoice Number**: INV_100000
- **Total Cost**: ₦5328.20
- **Due Date**: 5 days from today
- **Currency**: Nigerian Naira (NGN)
- **Logo**: Included from data/company_logo.png

You can download the invoice [here](styled_invoice.pdf). 

If you need any further assistance, feel free to ask!
